In [1]:
# Data Processing from base data to dataset for residential tilt analysis

In [2]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")

years = df_clean['Tax Year'].unique().tolist()
entities = df_clean['Entity Name'].unique().tolist()


In [3]:

# exclude personal property
df_clean['Total Taxable Property'] -= df_clean['Personal Property']

# drop needless columns
df_clean = df_clean[['Tax Year', 'County Name', 'Entity Name', 'Entity Code', 'County Code', 'Entity Type', 'Tax Rate' , 'Primary Residential', 
                 'Total Taxable Property', 'Primary Residential, Share', 'Revenue, Primary Residential', 'Revenue, Total']]

df_clean['Primary Residential, Share'] = df_clean['Revenue, Primary Residential']/ df_clean['Revenue, Total']


In [4]:
# Hold tax base shares constant over time

for entity in entities:
    df_clean.loc[(df_clean['Entity Name']== entity), 'Countertilt Share, Primary Residential'] = df_clean.loc[(df_clean['Entity Name']== entity) & (df_clean['Tax Year']== 2014), 'Primary Residential, Share'].sum()


# Compute Counterfactual revenue shares
df_clean['Countertilt Revenue, Primary Residential'] = df_clean['Revenue, Total'] * df_clean['Countertilt Share, Primary Residential']

# Compute Tilt Premium for Primary Residential Property
df_clean['Tilt Premium, Primary Residential']= df_clean['Revenue, Primary Residential'] - df_clean['Countertilt Revenue, Primary Residential']

In [5]:
# Export to JSON
df_clean.to_json('../../public/data/tilt_data.json', orient = 'records')
